# 1. Mathematical Formulation


## Subtask 1.1: Define the Classical Model

This subtask formally defines the classical objective function for the mean-variance portfolio optimization problem. The model is based on the foundational work of Markowitz, which posits a trade-off between portfolio return and portfolio risk.

### 1. Definition of Terms

First, we define the key mathematical and financial terms used in the model:

* **$n$**: The total number of available assets in the investment universe.
* **$x \in \{0, 1\}^n$**: The **decision variable vector**. This is a vector of $n$ binary variables, where $x_i = 1$ signifies that asset $i$ is selected for inclusion in the portfolio, and $x_i = 0$ signifies it is not selected.
* **$\mu \in \mathbb{R}^n$**: The **expected return vector**. This $n$-dimensional vector contains the expected annual return $\mu_i$ for each asset $i$.
* **$\Sigma \in \mathbb{R}^{n \times n}$**: The **covariance matrix**. This is an $n \times n$ symmetric matrix where the diagonal element $\Sigma_{ii}$ is the variance (individual risk) of asset $i$, and the off-diagonal element $\Sigma_{ij}$ is the covariance between asset $i$ and asset $j$, representing their co-movement.
* **$q \in \mathbb{R}^+$**: The **risk-aversion parameter**. This is a scalar constant (chosen by the investor) that quantifies the investor's tolerance for risk.
    * A **high $q$** value indicates a high aversion to risk, prioritizing risk minimization over return maximization.
    * A **low $q$** value indicates a low aversion to risk, prioritizing return maximization.

### 2. The Final Objective Function (to be Minimized)

The classical formulation seeks to find the portfolio $x$ that maximizes the risk-adjusted expected return. This is expressed as:

$$\text{Maximize:} \quad \mathbf{x}^T \boldsymbol{\mu} - q \cdot \mathbf{x}^T \mathbf{\Sigma} \mathbf{x}$$

Where:
* $\mathbf{x}^T \boldsymbol{\mu} = \sum_{i} x_i \mu_i$ represents the **Total Portfolio Return**.
* $\mathbf{x}^T \mathbf{\Sigma} \mathbf{x} = \sum_{i, j} x_i x_j \Sigma_{ij}$ represents the **Total Portfolio Risk (Variance)**.

However, standard optimization solvers (including those in Qiskit) are conventionally formulated as **minimization** problems. To align with this convention, we negate the entire function. Maximizing the original function is mathematically equivalent to minimizing its negative.

Therefore, the **final objective function to be minimized** is:

$$\text{Minimize:} \quad q \cdot \mathbf{x}^T \mathbf{\Sigma} \mathbf{x} - \mathbf{x}^T \boldsymbol{\mu}$$

This function provides the "score" for any given portfolio $x$. The goal of our quantum algorithm is to find the binary vector $x$ that results in the lowest possible value for this function, thereby identifying the optimal balance of risk and return as defined by the parameter $q$.


## Subtask 1.2: Define the Constraint

In addition to the objective function (Subtask 1.1), our optimization problem is subject to one or more **constraints**. A constraint is a rule or condition that the final solution *must* satisfy.

For this project, we are starting with the Minimal Viable Product (MVP) which simplifies the problem. A common constraint in portfolio optimization is a **budget constraint**, which dictates how many assets ($k$) should be selected from the total available pool ($n$).

### The MVP Budget Constraint

Our specific constraint is defined as follows:
> **The portfolio must contain *exactly* $k$ assets.**

For the purpose of our MVP, let's assume we are selecting $k=2$ assets from a total pool of $n=4$ assets.

Using our binary decision variable $x_i$ (where $x_i=1$ if asset $i$ is chosen, and $x_i=0$ if not), we can express this rule mathematically. The sum of all $x_i$ variables must equal our budget $k$.

### The Mathematical Equation

The mathematical equation for the budget constraint  is:

$$\sum_{i=1}^{n} x_i = k$$

**For our specific MVP (where $n=4$ and $k=2$), the equation is:**

$$\sum_{i=1}^{4} x_i = 2$$

Or, written out in full:

$$x_1 + x_2 + x_3 + x_4 = 2$$

This single, linear equation  precisely enforces our rule: any valid solution *must* have exactly two of the $x_i$ variables set to $1$, and the other two set to $0$.

Here is the professional, research-style write-up for Subtask 1.3.

---

## Subtask 1.3: Formulate the QUBO

A **QUBO (Quadratic Unconstrained Binary Optimization)** problem is a specific format required by many optimization solvers, including QAOA. The goal of this subtask is to convert our *constrained* problem (from 1.1 and 1.2) into a single *unconstrained* equation.

This involves two steps:
1.  Converting the budget constraint into a quadratic penalty term.
2.  Combining this penalty with the original objective function to create the final QUBO equation.

### 1. Converting the Constraint to a Penalty Term

Our constraint from Subtask 1.2 is a "hard" equality: $\sum_{i=1}^{n} x_i = k$.
To make this part of the objective function, we use the **penalty method**. We formulate a term that equals **zero** when the constraint is satisfied and a large **positive value** (a "penalty") when it is violated.

1.  **Re-frame the Constraint:** We start by re-framing the equation so that its target value is zero:
    $$\left(\sum_{i=1}^{n} x_i - k \right) = 0$$

2.  **Square the Term:** To ensure any deviation (both positive, e.g., $k+1$, and negative, e.g., $k-1$) results in a positive penalty, we square the entire expression:
  $$\left(\sum_{i=1}^{n} x_i - k \right)^2$$
    
    This expression is now **quadratic**.

3.  **Introduce Penalty Factor ($P$):** We multiply this quadratic term by a large positive constant, $P$. This $P$ (or "penalty factor") is a scalar value chosen to be large enough that any violation of the constraint adds a significant cost to the total objective, making such solutions highly unfavorable to the optimizer.

The final **quadratic penalty term** is:
$$P \left(\sum_{i=1}^{n} x_i - k \right)^2$$

###2. The Final Unconstrained QUBO Equation

We now create the final QUBO by combining our original objective function (Subtask 1.1) and the new penalty term (from above). The optimizer's goal is to minimize the sum of both parts.

The optimizer will be forced to satisfy the constraint (making the penalty term zero) to achieve a truly minimal value for the entire function.

**The final QUBO equation to be minimized is:**

$$\text{Minimize:} \quad \underbrace{\left( q \sum_{i,j} x_i x_j \Sigma_{ij} - \sum_{i} x_i \mu_i \right)}_{\text{Original Objective (Subtask 1.1)}} + \underbrace{P \left(\sum_{i=1}^{n} x_i - k \right)^2}_{\text{Penalty Term (from Constraint 1.2)}}$$

This single equation is now "unconstrained" because the constraint is embedded within it. It is "quadratic" due to the $\Sigma_{ij}$ and the squared penalty terms, and "binary" as $x_i \in \{0, 1\}$. This formulation is now ready for the next step: conversion to an Ising Hamiltonian.



## Subtask 1.4: Derive the Ising Hamiltonian

This subtask details the final mathematical translation of our problem. The QUBO formulation (Subtask 1.3) is a computational model common in computer science. To solve this using QAOA, we must map this QUBO to its equivalent **Ising Hamiltonian**, which is a physics-based model that quantum computers are naturally suited to solve.

The goal is to transform the QUBO, which uses binary variables $x_i \in \{0, 1\}$, into an Ising model, which uses spin variables $s_i \in \{-1, +1\}$.

### 1. Perform the Mathematical Substitution

This transformation is achieved using a standard substitution that maps the binary domain $\{0, 1\}$ to the spin domain $\{-1, +1\}$. Each binary variable $x_i$ is replaced with an expression involving a spin variable $s_i$:

$$x_i = \frac{s_i + 1}{2}$$

This substitution ensures that:
* When the spin $s_i = +1$ (spin up), $x_i = (+1 + 1) / 2 = 1$.
* When the spin $s_i = -1$ (spin down), $x_i = (-1 + 1) / 2 = 0$.

This substitution is applied to every $x_i$ and $x_i x_j$ term throughout the entire QUBO equation derived in Subtask 1.3.

### 2. Algebraically Expand and Simplify

Applying this substitution to our QUBO equation—which contains linear terms (from $\boldsymbol{\mu}$), quadratic terms (from $\boldsymbol{\Sigma}$), and the expanded quadratic penalty term—is a process of extensive algebraic expansion and simplification.

* **Linear terms** ($x_i$) become: $\frac{1}{2}s_i + \frac{1}{2}$
* **Quadratic terms** ($x_i x_j$) become: $\frac{1}{4}(s_i s_j + s_i + s_j + 1)$

When all terms from the objective function and the penalty are combined, expanded, and like terms are collected, the original QUBO is systematically transformed. The resulting classical Ising model has a specific, well-defined form:

$$H_{\text{classical}} = \sum_{i<j} J_{ij} s_i s_j + \sum_{i} h_i s_i + C$$

* **$J_{ij}$**: The **coupling coefficients** between spins $i$ and $j$. These are new constants derived from the $\Sigma_{ij}$ and $P$ terms.
* **$h_i$**: The **linear biases** (or external magnetic fields) acting on each spin $i$. These are new constants derived from $\mu_i$, $\Sigma_{ii}$, $P$, and $k$.
* **$C$**: A **constant energy offset**. This term collects all constants that do not depend on any $s_i$ variable. This offset shifts the entire energy spectrum but does **not** change the ground state (the optimal solution), so it can often be disregarded.

### The Final Ising Hamiltonian

The final step is to promote this classical Ising model to a quantum Hamiltonian. This is done by replacing the classical spin variables $s_i$ with their corresponding **Pauli-Z operators ($Z_i$)**, which act on qubit $i$.

The **final Ising Hamiltonian** $H$ to be solved by QAOA is:

$$H = \sum_{i<j} J_{ij} Z_i Z_j + \sum_{i} h_i Z_i + C \cdot I$$

Finding the ground state (lowest energy configuration) of this Hamiltonian $H$ is mathematically equivalent to finding the binary vector $x$ that minimizes our original QUBO. This $H$ is the precise input required by the QAOA algorithm.